In [9]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import random
import pickle
import neal


from PIL import Image as image

def loadData(filePath):
    f = open(filePath,'rb') 
    data= []
    
    img =image.open(f) 
    m,n =img.size
    img.show()
#     a = np.asarray(im)
    for i in range(m):
        for j in range(n):
            x =img.getpixel((i,j))
            data.append([x])
    f.close()
    return np.mat(data),m,n

imgData,row,col =loadData('1_small.png')
# imgData = imgData.reshape(row,col)

<frozen importlib._bootstrap>:228: RuntimeWarning: dimod.binary.cybqm.cybqm_float32.cyBQM_template size changed, may indicate binary incompatibility. Expected 64 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: dimod.binary.cybqm.cybqm_float32.cyBQM_float32 size changed, may indicate binary incompatibility. Expected 64 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: dimod.binary.cybqm.cybqm_float64.cyBQM_template size changed, may indicate binary incompatibility. Expected 64 from C header, got 112 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: dimod.binary.cybqm.cybqm_float64.cyBQM_float64 size changed, may indicate binary incompatibility. Expected 64 from C header, got 112 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: dimod.quadratic.cyqm.cyqm_float32.cyQM_template size changed, may indicate binary incompatibility. Expected 72 from C header, got 128 from PyObject
<frozen

In [8]:
!pip install dwave-neal

  Using cached dimod-0.12.13-cp39-cp39-macosx_10_9_x86_64.whl (5.3 MB)
  Attempting uninstall: dimod
    Found existing installation: dimod 0.10.17
    Uninstalling dimod-0.10.17:
      Successfully uninstalled dimod-0.10.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dwave-qbsolv 0.3.4 requires dimod<0.11.0,>=0.8.1, but you have dimod 0.12.13 which is incompatible.


In [145]:
N = imgData.shape[0]

In [227]:
from scipy import sparse as sp
from scipy import signal as sg

In [157]:
K2 = sp.csr_matrix(K)

In [259]:
K = np.zeros((N, N))

s_trash = 128

# заполнение квадратичной формы
for i in range(N):
    for j in range(N):
        
        if(i == j):
            
            K[i, i] += (imgData[i] - s_trash)**2
            
            if(i > 0):
                
                K[i, i] += (imgData[i] - imgData[i-1])**2
                
                
                
            if(i < N-1):
                
                K[i, i] += (imgData[i] - imgData[i+1])**2
                
            continue
            
        if(abs(i - j) == 1 or abs(i - j) == row):# or abs(i - j) == row + 1 or abs(i - j) == row - 1):
            
            K[i, j] = -1/2*(imgData[i] - imgData[j])**2
            

/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_22715/3725394333.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i, i] += (imgData[i] - s_trash)**2
/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_22715/3725394333.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i, i] += (imgData[i] - imgData[i+1])**2
/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_22715/3725394333.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i

In [260]:
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q=K, num_reads=10)

In [261]:
dict_out = list(sampleset.lowest().data())[0][0]
val_old = np.asarray(list(dict_out.values()))

In [262]:
val = val_old.reshape(row, col)
pic_new = image.new("L",(row,col))

for i in range(row):
    for j in range(col):
        pic_new.putpixel((i,j),int(256*(1-val[i][j])))
                         
pic_new.save("normal_name.jpg","JPEG")
pic_new.show()

In [245]:
val

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 1, 1]], dtype=int8)

In [121]:
%ls

1.png                  2_small_c.png          Task_3.ipynb
1_small.png            3.png                  bloch_inv.png
1_small_segmented.png  4.png                  grapes_inv.png
2.png                  KM_Apple.jpg           heraldic_inv.png


In [56]:
def d_circle(i, row, d):
    
    # возвращает индексы узлов в квадрате сполуребром = d
    
    ans = []
    
    # верхнее ребро
    ans += [k + i - d*row for k in list(range(-d, d+1))]
    
    # правое ребро
    ans += [k*row + i + d for k in list(range(-d, d+1))]
    
    # нижнее ребро
    ans += [-k + i + d*row for k in list(range(-d, d+1))]
    
    # левое ребро
    ans += [k*row + i - d for k in list(range(-d, d+1))]
    
    np.asarray(ans)
    
    return ans

In [79]:
%ls

1.png                  3.png                  grapes_inv.png
1_small.png            4.png                  heraldic_inv.png
1_small_segmented.png  KM_Apple.jpg           normal_name.jpg
2.png                  Task_3.ipynb
2_small_c.png          bloch_inv.png


In [52]:
d_circle(3, 4, 1)

[-2, -1, 0, 0, 4, 8, 8, 7, 6, -2, 2, 6]

In [50]:
for n in d_circle(3, 4, 1):
    
    print(n)

-2
-1
0
0
4
8
8
7
6
-2
2
6


In [242]:
# попытка через макскат

import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import random
import pickle
import neal
from scipy import sparse as sp
from scipy import signal as sg

from PIL import Image as image

def loadData(filePath, conv=1):
    f = open(filePath,'rb') 
    data= []
    
    img =image.open(f) 
    m,n =img.size
    img.show()
    
    tmp = np.zeros((m, n))
    
    for i in range(m):
        for j in range(n):
            tmp[i, j] = img.getpixel((i,j))
    f.close()
    
    data = sg.convolve2d(tmp, np.ones((conv, conv)))/conv**2
    
    row = data.shape[0]
    col = data.shape[1]
    
    pic_new = image.new("L",(row,col))
    
    for i in range(row):
        for j in range(col):
            pic_new.putpixel((i,j), int(data[i, j]))
    
    pic_new.show()
    
    return np.mat(data),row,col

In [243]:
imgData,row,col =loadData('1_small.png', conv=3)
# imgData = imgData.reshape(row,col)


N = imgData.shape[0]


In [218]:
w_1 = 1/256*0.
w_2 = 1/256*0.16


from scipy import sparse as sp

#K = sp.lil_matrix((N, N), dtype=np.int16)
#K = sp.csr_matrix(K)
K = np.zeros((N, N))

# заполнение квадратичной формы
for i in range(N):
    
    #print(i, '\n')
    neighbors = d_circle(i, row, d=2)
    
    for n in neighbors:
        
        if(n >= 0 and n < N):

            tmp = 1 - w_2*(imgData[i] - imgData[n])**2

            K[i, i] += int(tmp)
            K[i, n] -= int(tmp)

    neighbors = d_circle(i, row, d=1)
    
    for n in neighbors:
        
        if(n >= 0 and n < N):

            tmp = 1 - w_1*(imgData[i] - imgData[n])**2

            K[i, i] += int(tmp)
            K[i, n] -= int(tmp)

/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_29716/150097380.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i, i] += int(tmp)
/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_29716/150097380.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i, n] -= int(tmp)
/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_29716/150097380.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i, i] += int(tmp)
/var/folders/d0/tt3hk6g15

In [193]:
# ручное
# w = 1/256*0.16

# K = np.zeros((N, N))

# # заполнение квадратичной формы
# for i in range(N):
            
#     if(i - 1 >= 0):

#         tmp = 1 - w*(imgData[i] - imgData[i - 1])**2
        
#         K[i, i] += tmp
#         K[i, i - 1] -= tmp

#     if(i + 1 < N):

#         tmp = 1 - w*(imgData[i] - imgData[i + 1])**2
        
#         K[i, i] += tmp
#         K[i, i + 1] -= tmp

#     if(i - row >= 0):
            
#         tmp = 1 - w*(imgData[i] - imgData[i - row])**2
        
#         K[i, i] += tmp
#         K[i, i - row] -= tmp

#     if(i + row < N):
        
#         tmp = 1 - w*(imgData[i] - imgData[i + row])**2
        
#         K[i, i] += tmp
#         K[i, i + row] -= tmp

#     if(i - 1 - row >= 0):
        
#         tmp = 1 - w*(imgData[i] - imgData[i - 1 - row])**2
        
#         K[i, i] += tmp
#         K[i, i - 1 - row] -= tmp

#     if(i + 1 - row >= 0):
            
#         tmp = 1 - w*(imgData[i] - imgData[i + 1 - row])**2
        
#         K[i, i] += tmp
#         K[i, i + 1 - row] -= tmp

#     if(i + 1 + row < N):
              
#         tmp = 1 - w*(imgData[i] - imgData[i + 1 + row])**2
        
#         K[i, i] += tmp
#         K[i, i + 1 + row] -= tmp

#     if(i - 1 + row < N):

#         tmp = 1 - w*(imgData[i] - imgData[i - 1 + row])**2
        
#         K[i, i] += tmp
#         K[i, i - 1 + row] -= tmp


In [195]:
%ls

1.png                  4.png                  grapes_inv.png
1_small.png            KM_Apple.jpg           heraldic_inv.png
1_small_segmented.png  Q3.pkl                 normal_name.jpg
2.png                  Q3sp.pkl               solution.yaml
2_small_c.png          Task_3.ipynb
3.png                  bloch_inv.png


In [226]:
a = open('solution.yaml')

False

In [220]:
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q=K, num_reads=50)
dict_out = list(sampleset.lowest().data())[0][0]
val_old = np.asarray(list(dict_out.values()))

val = val_old.reshape(row, col)
pic_new = image.new("L",(row,col))

for i in range(row):
    for j in range(col):
        pic_new.putpixel((i,j),int(256*(1-val[i][j])))
                         
pic_new.save("normal_name.jpg","JPEG")
pic_new.show()

In [128]:
# копипаста

import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import random
import pickle
import neal

from PIL import Image as image

def loadData(filePath):
    f = open(filePath,'rb') 
    data= []
    
    img =image.open(f) 
    m,n =img.size
    img.show()
#     a = np.asarray(im)
    for i in range(m):
        for j in range(n):
            x =img.getpixel((i,j))
            data.append([x])
    f.close()
    
    return np.mat(data),m,n

img,row,col =loadData('1_small.png')
# imgData = imgData.reshape(row,col)

img = img.reshape((row, col))

def unary_potential(img,label,i,j):
	if(label == 0):
		return (img[i,j]/255.0) - 1
	else:
		return (0 - (img[i,j]/255.0))

def find_neighbors(img,i,j):
	(M,N)=img.shape[0:2]
	#find correct neighbors
	
	#right-bottom corner(no post neighbors available for this pixel)
	if i==M-1 and j==N-1:
		neighbor=[]
	#bottom line
	elif i==M-1:
		neighbor=[(M-1,j+1)]
	#right line
	elif j==N-1:
		neighbor=[(i+1,N-1)]
	#any other pixel
	else:
		neighbor=[(i,j+1), (i+1,j+1), (i+1,j)]

	return neighbor

(M,N) = img.shape[0:2]
coupler_count = 0
Q = {}
for i in range(M):
    for j in range(N):
        foreground_node_id = (i*N*2)+(j*2)
        background_node_id = foreground_node_id+1
        #foreground qubit
        Q[(foreground_node_id,foreground_node_id)] = unary_potential(img,1,i,j)
        #background qubit
        Q[(background_node_id,background_node_id)] = unary_potential(img,0,i,j)
        #for qubits of same pixel - high cost should be given here to ensure both qubits are not open
        Q[(foreground_node_id,background_node_id)] = 10
        coupler_count += 1
        #for neighbors
        neighbors = find_neighbors(img,i,j)
        for k in neighbors:
            neighbor_foreground_node_id = (k[0]*N*2)+(k[1]*2)
            neighbor_background_node_id = neighbor_foreground_node_id+1


#print(Q)			

sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q=Q, num_reads=100)
dict_out = list(sampleset.lowest().data())[0][0]
array = np.asarray(list(dict_out.values()))

counter = 1
out = []
for i in array:
    mode = counter
    if(counter > (N*M*2)):
        continue
    if(mode%2 == 1):
        if array[i] == 1:
            out.append(int(255))
        else:
            out.append(0)
    counter += 1


out=np.array(out)
val = out.reshape(M,N)

pic_new = image.new("L",(row,col))

for i in range(row):
    for j in range(col):
        pic_new.putpixel((i,j),int(256*(1-val[i][j])))
                         
pic_new.save("normal_name.jpg","JPEG")
pic_new.show()

In [184]:
N

37200

In [158]:
with open("Q3sp.pkl", "wb") as fout:
    pickle.dump(K2, fout)

In [154]:
K.shape

(37200, 37200)